**Calculation Contexts**

Usually, when evaluating functions a user often supplies additional arguments, such as date range, type of things to be included etc.

These additional arguments are supplied to IntelligentGraph via an object called an 'EvaluationContext'. An EvaluationContext contains a 'bag' of name/value pairs called customQueryOptions which can be accessed within any script running within the same context using a call like this: "var localtime = $customQueryOptions.get("time");"  where "time" in this case is the name of the object included in the customQueryOptions bag.

To demonstrate this lets connect to our graph and add a claculation that depends on a customQueryOption

In [1]:
%jars /usr/share/olgap-0.1.0.jar

You will also need to 'import' the classes that you will use in your code:

In [2]:
import pathCalc.Thing;
import pathQLModel.Resource;
import pathCalc.Evaluator;
import pathQLRepository.*;

You need to access an existing IntelligentGraph repository, and in this example assign it to the variable 'source'

In [3]:
PathQLRepository source = PathQLRepository.create("http://host.docker.internal:8080/rdf4j-server","calc2graph");

Carefully note the use of 'http://host.docker.internal:8080'. This is the equivalent of 'http://localhost:8080' when Jupyter is running in a Docker container accessing a Repository on 'http://localhost:8080'

Unless you like typing in the full IRI  (aka URL) of 'things, it is a good idea to define some prefixes so we can use qNames instead:

In [4]:
source.prefix("<http://inova8.com/calc2graph/def/>");
source.prefix("rdfs","<http://www.w3.org/2000/01/rdf-schema#>");

pathQLRepository.PathQLRepository@4a4c1e06

Just to make sure we are stating with a clean sheet, lets remove the graph:

In [5]:
source.removeGraph("<http://inova8.com/calc2graph/contextGraph>");

http://inova8.com/calc2graph/contextGraph

In [6]:
Graph graph = source.addGraph("<http://inova8.com/calc2graph/contextGraph>");

As before let's work with a myCountry 'thing'

In [7]:
Thing myCountry = graph.getThing(":Country1");

This time we will only add a calculated fact, and all this calculation does is access a customQueryOption value

In [8]:
String customQueryOptionTimeScript = "$customQueryOptions.get(\"time\");";
myCountry.addFact(":time", customQueryOptionTimeScript, Evaluator.GROOVY) ;

2021-05-20 17:34:37 INFO  Evaluator- 3 scripting engines were found with the following short names:js;JS;JavaScript;javascript;ECMAScript;ecmascript;Graal.js;graal.js;Graal-js;graal-js;Graal.JS;Graal-JS;GraalJS;GraalJSPolyglot;groovy;Groovy;python;jython;


http://inova8.com/calc2graph/def/Country1

In [9]:
//myCountry.getFact(":time")

So why did we get an error? Because we have not yet initialized any customQueryOptions, so IntelligentGraph just treats it as a NULL value.

In [10]:
import pathCalc.CustomQueryOptions;

CustomQueryOptions  customQueryOptions = new CustomQueryOptions();
customQueryOptions.put("time",42);
customQueryOptions.put("name","Peter");
customQueryOptions;

{name="Peter", time="42"^^<http://www.w3.org/2001/XMLSchema#int>}

And we can see from the following the customQueryOptions 'bag' is empty

In [11]:
myCountry = graph.getThing(":Country1",customQueryOptions );

In [14]:
myCountry.getFact(":time");

"Evaluated null for http://inova8.com/calc2graph/def/time of http://inova8.com/calc2graph/def/Country1, using script "$customQueryOptions.get("time");"^^<http://inova8.com/script/groovy>"